In [6]:
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from prepare.eegdataset import N_Mix_GeneralEEGImageDataset, GeneralEEGImageDataset, N_GeneralEEGImageDataset, N2_GeneralEEGImageDataset
import numpy as np
import matplotlib.pyplot as plt
import torch
data_path = '/data0/tianjunchao/dataset/CVPR2021-02785/data/img_pkl/32x32'
# dataset = GeneralEEGImageDataset(path=data_path, n_channels=1, grid_size=8)
dataset = N_Mix_GeneralEEGImageDataset(path=data_path, n_channels=1, grid_size=8)

dataloader = DataLoader(dataset, batch_size=32)
# def a function to show images
import os

from utils.eegutils import getNow
from run.others import SmallNet
from prepare.data import filter_two_samples
import run.resnet as resnet
import torch.nn as nn

material_dir = "4.materials/features/"+getNow()+"/"
if not os.path.exists(material_dir):
    os.makedirs(material_dir)

In [7]:
def show_images(imgs, labels,img_dir=None):
    # 将通道维度放在最后，以符合imshow的要求
    imgs = np.transpose(imgs, (0, 2, 3, 1))

    n_imgs = len(labels)
    ncols = 4
    nrows = n_imgs//ncols

    # print('imgs.shape: ', imgs.shape)
    # 打印16张图像
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(10, 10))
    # 显示每个图像的标签


    for i, ax in enumerate(axes.flat):
        ax.set_title(labels[i])
        ax.imshow(imgs[i])
        ax.set_axis_off()

    # 调整子图之间的间距
    plt.tight_layout()

    if img_dir is not None:
    # 显示图像
    # plt.show()
        plt.savefig(img_dir+ str(labels[0])+'-'+str(labels[len(labels)-1])+'.png')
        plt.close()

def show_channels(img):
    plt.imshow(img)

    # 显示图像
    plt.show()
    # 分别画出每个通道的图像
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(9, 3))

    for i, ax in enumerate(axes):
        ax.imshow(img[..., i])
        ax.set_title('Channel {}'.format(i+1))

    # 调整子图之间的间距
    plt.tight_layout()

    # 显示图像
    plt.show()



# draw images in each segment
def draw_list(segment):
    images = []
    labels = []

    # 遍历segment中的每个元素
    for i, data in enumerate(segment):
        x1,x2, y = data[0], data[1],data[2]
        
        images.append(x1)
        images.append(x2)
        # append y to labels
        labels.append(y)
        labels.append(y)
    
    images = torch.stack(images, dim=0).view(len(labels), 3, 224, 224)
    return images,labels

In [8]:
result_images = []
result_labels = []

for i, (raw_images, raw_labels) in enumerate(dataloader):
    if i == 2:
        break
    # raw images is a list with 2 elements, each element is a tensor of shape (batch_size, 3, 224, 224), i want to get batch, 2, 3, 224, 224
    n_samples = len(raw_images)
    # print('n_samples: ', n_samples)
    raw_images = torch.stack(raw_images, dim=1)
    # # dimension of raw-labels is [2,], copy it to dimension [2,16]
    raw_labels = raw_labels.unsqueeze(1).repeat(1,n_samples)
    # print('raw_images.shape: ', raw_images.shape)
    # print('raw_labels.shape: ', raw_labels.shape)
    # get 32x4,3,224,224
    raw_images = raw_images.view(-1, 3, 224, 224)
    # get 32x4
    raw_labels = raw_labels.view(-1)
    result_images.append(raw_images)
    result_labels.append(raw_labels)
    print('i:',i)

result_images = torch.stack(result_images, dim=0)
result_labels = torch.stack(result_labels, dim=0)
print(result_images.shape)
print(result_labels.shape)


i: 0
i: 1
torch.Size([2, 128, 3, 224, 224])
torch.Size([2, 128])


In [9]:
# 用pca降维为50维度，然后用tsne降维为2维度
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE


def get_tsne_features(features):
    features = PCA(n_components=50).fit_transform(features)
    features = TSNE(verbose=1,learning_rate='auto',init='pca').fit_transform(features)
    return features

In [10]:
model_list = [resnet.resnet_name_A,resnet.resnet_name_B,resnet.resnet_name_C,resnet.resnet_name_D,resnet.resnet_name_E]
# model_list = ['smallnetA']
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
for model_name in model_list:
    if 'resnet' in model_name:
        model = list(resnet.ResNet(num_classes=40,model_name=model_name, pretrained=True).children())[0]
    else:
        model = list(SmallNet().children())[0]

    feature_extractor = nn.Sequential(*list(model.children())[:-1])
    
    if 'mobilenet' in model_name:
        feature_extractor.add_module("AdaptiveAvgPool", nn.AdaptiveAvgPool2d(1))

    feature_extractor = feature_extractor.to(device)
    
    feature_extractor.eval() 
    print(model_name)


    
    feature_stack = []
    label_stack = []

    for i in range(result_images.shape[0]):
        
        images = result_images[i].to(device)
        with torch.no_grad():
            # for img in images:
            features = feature_extractor(images).squeeze()
            feature_stack.append(features)
            label_stack.append(result_labels[i])
            
        
    feature_list = torch.cat(feature_stack, dim=0)
    label_list = torch.cat(label_stack, dim=0)
    print('feature_list.shape: ', feature_list.shape)
    print('label_list.shape: ', label_list.shape)
    tsne_features = get_tsne_features(feature_list.cpu().numpy())
    
    # plot tsne and save image
    fig = plt.figure(figsize=(10, 10))
    plt.scatter(tsne_features[:, 0], tsne_features[:, 1], c=label_list, cmap='tab10')
    plt.colorbar()
    plt.title('t-SNE features -' + model_name)
    # plt.show()
    plt.savefig(material_dir+ model_name+'tsne.png')
    plt.close()

resnet18
feature_list.shape:  torch.Size([256, 512])
label_list.shape:  torch.Size([256])
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 256 samples in 0.000s...
[t-SNE] Computed neighbors for 256 samples in 0.028s...
[t-SNE] Computed conditional probabilities for sample 256 / 256
[t-SNE] Mean sigma: 3.566378


/data0/tianjunchao/anaconda3/envs/tian/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:982: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


[t-SNE] KL divergence after 250 iterations with early exaggeration: 54.308838
[t-SNE] KL divergence after 1000 iterations: 0.475743
resnet34
feature_list.shape:  torch.Size([256, 512])
label_list.shape:  torch.Size([256])
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 256 samples in 0.000s...
[t-SNE] Computed neighbors for 256 samples in 0.029s...
[t-SNE] Computed conditional probabilities for sample 256 / 256
[t-SNE] Mean sigma: 5.001968


/data0/tianjunchao/anaconda3/envs/tian/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:982: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


[t-SNE] KL divergence after 250 iterations with early exaggeration: 54.437443
[t-SNE] KL divergence after 1000 iterations: 0.459844
resnet50
feature_list.shape:  torch.Size([256, 2048])
label_list.shape:  torch.Size([256])
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 256 samples in 0.000s...
[t-SNE] Computed neighbors for 256 samples in 0.025s...
[t-SNE] Computed conditional probabilities for sample 256 / 256
[t-SNE] Mean sigma: 2.081969


...

[t-SNE] KL divergence after 250 iterations with early exaggeration: 53.944016
[t-SNE] KL divergence after 1000 iterations: 0.551060
resnet101
feature_list.shape:  torch.Size([256, 2048])
label_list.shape:  torch.Size([256])
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 256 samples in 0.000s...
[t-SNE] Computed neighbors for 256 samples in 0.026s...
[t-SNE] Computed conditional probabilities for sample 256 / 256
[t-SNE] Mean sigma: 3.451401


/data0/tianjunchao/anaconda3/envs/tian/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:982: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


[t-SNE] KL divergence after 250 iterations with early exaggeration: 54.157444
[t-SNE] KL divergence after 1000 iterations: 0.475561
resnet152
feature_list.shape:  torch.Size([256, 2048])
label_list.shape:  torch.Size([256])
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 256 samples in 0.000s...
[t-SNE] Computed neighbors for 256 samples in 0.022s...
[t-SNE] Computed conditional probabilities for sample 256 / 256
[t-SNE] Mean sigma: 2.761851


/data0/tianjunchao/anaconda3/envs/tian/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:982: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


[t-SNE] KL divergence after 250 iterations with early exaggeration: 53.866749
[t-SNE] KL divergence after 1000 iterations: 0.406061
